In [36]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
from metric import compute_metric
import pickle
import pandas as pd
import matplotlib.pyplot as plt


runtime = '2023-09-26_23-20-59'
thres = 0.5
plt.figure()
idx = 4
data_dir = f'../datasets/Case-study/binary.tsv'
name = f'model{idx}'
pkl_path = f'../result/binary/{runtime}/{name}'
results = sorted(os.listdir(pkl_path))
num_split = len(results)
dataset = pd.read_csv(data_dir, sep='\t')
dataset['seqlen'] = dataset['text'].apply(lambda x: len(x))
for i in range(num_split):
    pkl_dir = f'{pkl_path}/{results[i]}'
    with open(pkl_dir, 'rb') as fr:
        pkl = pickle.load(fr)
        outputs, labels = pkl
        compute_metric(labels[:, 1], outputs[:, 1], thres=thres)
    dataset[i] = outputs[:, 1]
column_names = range(num_split)
averages = dataset[column_names].mean(axis=1)
print('Mean')
auroc_val, fpr, tpr = compute_metric(labels[:, 1], averages, thres=thres)
dataset['avg'] = averages

0.9309	1.0	nan	0.0	nan
0.9309	1.0	nan	0.0	nan
0.9355	1.0	nan	0.0	nan
0.9447	1.0	nan	0.0	nan
0.9493	1.0	nan	0.0	nan
Mean
0.9493	1.0	nan	0.0	nan


/data/s853wang/ACP/DUO-ACP/notebook/../metric.py:26: RuntimeWarning: invalid value encountered in scalar divide
  specificity = tn / (tn + fp)
/data/s853wang/miniconda3/envs/ACP/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1123: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
/data/s853wang/ACP/DUO-ACP/notebook/../metric.py:26: RuntimeWarning: invalid value encountered in scalar divide
  specificity = tn / (tn + fp)
/data/s853wang/miniconda3/envs/ACP/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1123: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
/data/s853wang/ACP/DUO-ACP/notebook/../metric.py:26: RuntimeWarning: invalid value encountered in scalar divide
  specificity = tn / (tn + fp)
/data/s853wang/miniconda3/envs/ACP/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1123: UndefinedMetricWarning: No negative samples in y_t

<Figure size 640x480 with 0 Axes>

In [38]:
from metric import compute_metric_labelwise, compute_metric_score
import numpy as np
import pickle

folder = f'../result/multiclass/{runtime}'
labels, outputs = [], []
for k in range(1, 11):
    result_file = f'{folder}/model3/{k}.pkl'
    res = pickle.load(open(result_file, 'rb'))
    output, label = res
    labels.append(label)
    outputs.append(output)

labels = np.mean(labels, axis=0)
outputs = np.mean(outputs, axis=0)
print(outputs.shape)

#outputs
compute_metric_labelwise(labels, outputs)
compute_metric_score(labels, outputs)

(217, 7)
0.924	0.802	0.765	0.568	0.852	0.598
0.908	0.834	0.925	0.853	0.872	0.646
0.953	0.876	0.858	0.837	0.884	0.752
0.962	0.889	0.882	0.896	0.882	0.778
0.973	0.889	0.914	0.915	0.889	0.78
0.965	0.908	0.898	0.929	0.888	0.81
0.948	0.797	0.29	0.779	0.45	0.476
Average
0.948	0.856	0.79	0.825	0.817	0.691
------Macro------
Macro accuracy 0.8564845292955893
Macro precision 0.7904665672869388
Macro recall 0.9033205249114415
Macro f1-score 0.8167966505264631
------Micro------
Micro accuracy 0.8564845292955892
Micro precision 0.8199481865284974
Micro recall 0.8890449438202247
Micro f1-score 0.8530997304582211
------Hamming loss------
Hamming loss 0.14351547070441079


0.8564845292955893

In [40]:
from pyteomics import fasta
from util import all_labels
fa = fasta.read('../datasets/Case-study/multiclass.fasta')
seq_list = []
for x in fa:
    seq_list.append(x.sequence)
    
df = pd.DataFrame(outputs, columns=all_labels)
df['seq'] = seq_list
df['binary'] = averages
df = df.sort_values('binary', ascending=True).head(30)
df

,Colon,Breast,Cervix,Lung,Skin,Prostate,Blood,seq,binary
20,0.910784,0.902914,0.075354,0.286642,0.524923,0.206589,0.149712,DEVD,0.040214
190,0.955908,0.196706,0.108218,0.371810,0.627234,0.268454,0.072202,PPPEE,0.144144
183,0.969479,0.127177,0.908267,0.178633,0.102099,0.022082,0.416641,VNWOOILGOIIOVVO,0.197288
197,0.966335,0.103362,0.157802,0.025052,0.034278,0.006508,0.082401,QETFSDLWKLLP,0.199104
188,0.888724,0.196118,0.091182,0.009620,0.019230,0.008007,0.044612,LTFEHYWAQLTS,0.219841
18,0.101857,0.904682,0.082214,0.031209,0.088076,0.060804,0.135007,ALWKDILKNVGKAAGKAVLNTVTDMVNQ,0.245943
133,0.870959,0.861498,0.844574,0.778185,0.032613,0.056338,0.111741,RGDLLRHVVKILEKYL,0.262545
189,0.955509,0.038473,0.070034,0.006660,0.069249,0.009460,0.730452,PDEDAINNALNKVCSTGRRQRSICKQLLKK,0.292855
206,0.623079,0.650247,0.064718,0.174503,0.326181,0.234098,0.218707,ERRP,0.335832
106,0.137346,0.075570,0.092394,0.091258,0.216821,0.006295,0.437847,KSCCPNTTGRNIYNTCRFGGGSREVCARISGCKIISASTCPSDYPK,0.404619
